# Cria Tabelas

## Bibliotecas

In [1]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns; sns.set()
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
import camelot
import folium
from folium import plugins
import pathlib

pd.options.display.float_format = '{:.2f}'.format
home = Path.home()
parent = pathlib.Path().absolute()
cidade='piracicaba'
path_ipca_midlle = Path(parent, "bases_brutas/ipca/ipca2.csv")

path_download2 = Path(parent, "bases_brutas/ipca/ipca.csv")

path_download3 = Path(parent,"bases_brutas/{}/prefeitura/receitas/receita_bimestres_int.csv".format(cidade))

path_download4 = Path(parent,"bases/{}/receita_bimestres.csv".format(cidade))

path_download5 =  Path(parent,"bases/{}/receita_ano_pct.csv".format(cidade))

path_download6 = Path(parent,"bases_brutas/{}/prefeitura/despesas/despesa_bimestres_int.csv".format(cidade))

path_download7 = Path(parent,"bases/{}/despesa_bimestres.csv".format(cidade))

path_download8 = Path(parent,"bases/{}/despesa_ano_pct.csv".format(cidade))

path_download9 = Path(parent,"bases_brutas/{}/prefeitura/despesas/despesa_bimestre_funcao_temp.csv".format(cidade))

path_download10 = Path(parent,"bases/{}/".format(cidade))

path_download11 = Path(parent,"bases_brutas/{}/eleicao/vereadores/votacao.csv".format(cidade))

path_download12 = Path(parent,"bases_brutas/{}/eleicao/vereadores/sessoes_pira.csv".format(cidade))

## Funções

In [2]:
# Função retorna o mes escrito:
def mes(x):
    if x == '01':
        return 'fevereiro '
    elif x == '02':
        return 'abril '
    elif x == '03':
        return 'junho '
    elif x == '04':
        return 'agosto '
    elif x == '05':
        return 'outubro '
    else:
        return 'dezembro '

# Função retorna o mes do bimestre:
def mes_ind(x):
    if x == '01':
        return '02'
    elif x == '02':
        return '04' 
    elif x == '03':
        return '06'
    elif x == '04':
        return '08'
    elif x == '05':
        return '10'
    else:
        return '12'
    
# Função que arruma as datas da df para formato timestamp:
def arruma_data(df):
    df['Data'] = df['Bimestre'].apply(mes)
    df['tempo'] = df['Bimestre'].apply(mes_ind)
    df['tempo'] = df['tempo']+df['Ano']
    df['tempo'] = pd.to_datetime(df['tempo'])
    df['Data'] = df['Data'] + df['Ano']
    df.set_index('Data',inplace=True)
    return df
    
# Função que arruma os numeros  do df 'RECEITA' para formato float:
def arruma_num_rec(df):
    for i in ['Previsao Inicial', 'Previsao Atualizada (a)',
       'Receitas Realizadas no Bimestre (b)', '% (b/a)',
       'Receitas Realizadas até o Bimestre (c)', '% (c/a)', 'SALDO (a-c)']:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df

# Função que arruma os numeros  do df 'DESPESAS' para formato float:
def arruma_num_des(df):
    colunas=['DOTAÇÃO INICIAL (d)',
       'DOTAÇÃO ATUALIZADA (e)', 'DESPESAS EMPENHADAS NO BIMESTRE',
       'DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)', 'SALDO (g) =(e-f)',
       'DESPESAS LIQUIDADAS NO BIMESTRE',
       'DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)', 'SALDO (i) = (e-h)',
       'DESPESAS PAGASATÉ O BIMESTRE(j)']
    for i in colunas:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: str(x))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df

# Função que arruma os numeros  do df 'DESPESA-FUNCAO' para formato float:
def arruma_num_des_fun(df):
    colunas=['DOTAÇÃO INICIAL (d)','DOTAÇÃO ATUALIZADA (e)','DESPESAS EMPENHADAS NO BIMESTRE',
              'DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)',('% (b/total b)'),'SALDO (g) =(e-f)','DESPESAS LIQUIDADAS NO BIMESTRE',
              'DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)','% (d/total d)','SALDO (i) = (e-h)','DESPESAS PAGASATÉ O BIMESTRE(j)']
    for i in colunas:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: str(x))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df


# Função que arruma os numeros unicos para formato float:
def arruma_numero_unico(num):
    num= num.replace('.','')
    num= num.replace(',','.')
    num= float(num)
    return num

# Função que aplica a inflação no df RECEITA:
def aplica_inflacao_rec(filtered_df):
    for i in [1,2,3,5,7]:
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]/filtered_df.iloc[j,11])*100
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que aplica a inflação no df DESPESA:
def aplica_inflacao_des(filtered_df):
    colunas=[1,2,3,4,5,6,7,8,9]
    for i in colunas:
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]/filtered_df.iloc[j,13])*100
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que aplica a inflação no df DESPESA-FUNÇÃO:
def aplica_inflacao_des_fun(filtered_df):
    colunas=[1,2,3,4,7,8]
    for i in colunas:
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]*100)/filtered_df.iloc[j,14]
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que substitui '-' por 0:
def acha_string(x):
    if x=='-':
        return 0
    else:
        return int(x)

# Função que arruma no df VEREADORES
def numeros_sessoes(x):
    sessoes=[]
    lista_sessoes=x.split(';')
    for i in range(len(lista_sessoes)):
        if len(lista_sessoes[i].split())>=4:
            sessoes.extend(list(range(int(lista_sessoes[i].split()[1].replace('ª','').replace(' ','')),int(lista_sessoes[i].split()[3].replace('ª','').replace('.','').replace(' ',''))+1)))
        else:
            sessoes.append(lista_sessoes[i].replace('ª','').replace(' ',''))
    return sessoes

def retorna_posicao(x):
    
    for i in range(len(locais['Seções2'])):
        if str(x) in locais['Seções2'].iloc[i]:
            return locais['Latitude'].iloc[i]
        if x in locais['Seções2'].iloc[i]:
            return locais['Latitude'].iloc[i]
    return 0


def votos_totais_secao(x):
    for i in range(len(df.Seção)):
        if x == df.Seção.iloc[i]:
            return df.Votos.iloc[i]
    return -1

def modulo(x):
    if x > 0:
        return x
    else:
        return x*(-1)
    
def eh_par(x):
    if x % 2 == 0:
        return 1
    else:
        return 0

## IPCA

### Download da base bruta

#### OBS: devemos modificar o arquivo para excel e mudar o nome para ipca2

In [38]:
#*************MODIFICAR O path_download PARA A PASTA DE DOWNLOAD DO SEU PC*************

here = pathlib.Path().absolute()

path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)

link = 'https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas'
print(link)
driver.get(link)

https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas


### Rodar quando já tiver a base instalada

In [3]:
#Abre a base de dados baixada
ipca = pd.read_csv(path_ipca_midlle,';')
ipca=ipca[:2]
ipca.reset_index(inplace=True)
ipca.columns=ipca.iloc[0]
ipca=ipca.iloc[1:].T

#Arruma colunas
ipca.columns = ipca.iloc[0]
ipca = ipca.iloc[1:]
ipca.dropna(axis=0,inplace=True)
ipca = ipca.loc['fevereiro 2016':]
ipca['Brasil']=ipca['Brasil'].astype(float)
ipca['Outro']=ipca['Brasil'].shift(-1)
ipca['Acumulado'] = [0]*len(ipca['Brasil'])
ipca.iloc[-1,0]=100
df1=ipca

i=-1
while i >= -len(df1['Brasil']):
    ant=i+1
    if i==-1:
        df1.iloc[i , 2]=100
    else:
        df1.iloc[i , 2]=(100/(100+df1.iloc[i , 1])*df1.iloc[ant , 2])
    i=i-1

df1.drop(['Brasil','Outro'],axis=1,inplace=True)
df1.columns = ['base = ultimo periodo']
df1.reset_index(inplace=True)


#Arruma as datas
meses = ['fevereiro','abril','junho','agosto','outubro','dezembro']
ano = ['2016','2017','2018','2019','2020']
lista = []
for i in ano:
    for j in meses:
        data="{} {}".format(j,i)
        lista.append(data)
df1 = df1[df1[0].isin(lista)]
if len(df1[0])%2:
    df1.set_index(0,inplace=True)
else:
    df1 = df1.iloc[:-1] #ALTERAR AQUI A DATA PARA OUTROS MESES
    df1.set_index(0,inplace=True)
#Exporta os dados
ipca=df1
ipca.to_csv(path_download2, index = True)
ipca

,base = ultimo periodo
0,
fevereiro 2016,82.57
abril 2016,83.43
junho 2016,84.37
agosto 2016,85.18
outubro 2016,85.47
dezembro 2016,85.89
fevereiro 2017,86.50
abril 2017,86.83
junho 2017,86.90


## Receitas por bimestre

In [3]:
bimestre = ['01','02','03','04','05','06']
ano = ['2020','2019','2018','2017','2016']
receita_orcamentaria=pd.DataFrame()
for j in ano:
    for i in bimestre:
        print("Começando a processar o bimestre {}/{}".format(i,j))
        if int(j) ==2020:
            if int(i)==6:
                continue
            else:
                pass
        
        file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        todo = camelot.read_pdf(file,pages='1-2')

        receita_orcamentaria_temp=todo[1].df.iloc[4:]
        receita_orcamentaria_temp=receita_orcamentaria_temp.append(todo[3].df.iloc[3:])
        receita_orcamentaria_temp.columns = ['Receitas Orçamentárias','Previsao Inicial','Previsao Atualizada (a)','Receitas Realizadas no Bimestre (b)','% (b/a)','Receitas Realizadas até o Bimestre (c)','% (c/a)','SALDO (a-c)']
        receita_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(receita_orcamentaria_temp['Receitas Orçamentárias'])
        receita_orcamentaria_temp['Bimestre'] = ['{}'.format(i)]*len(receita_orcamentaria_temp['Receitas Orçamentárias'])
        receita_orcamentaria_temp['Ano'] = ['{}'.format(j)]*len(receita_orcamentaria_temp['Receitas Orçamentárias'])
        receita_orcamentaria = pd.concat([receita_orcamentaria_temp,receita_orcamentaria])
        
receita_orcamentaria = arruma_data(receita_orcamentaria)
receita_orcamentaria.to_csv(path_download3, index = True)



Começando a processar o bimestre 01/2020
Começando a processar o bimestre 02/2020
Começando a processar o bimestre 03/2020
Começando a processar o bimestre 04/2020
Começando a processar o bimestre 05/2020
Começando a processar o bimestre 06/2020
Começando a processar o bimestre 01/2019
Começando a processar o bimestre 02/2019
Começando a processar o bimestre 03/2019
Começando a processar o bimestre 04/2019
Começando a processar o bimestre 05/2019
Começando a processar o bimestre 06/2019
Começando a processar o bimestre 01/2018
Começando a processar o bimestre 02/2018
Começando a processar o bimestre 03/2018
Começando a processar o bimestre 04/2018
Começando a processar o bimestre 05/2018
Começando a processar o bimestre 06/2018
Começando a processar o bimestre 01/2017
Começando a processar o bimestre 02/2017
Começando a processar o bimestre 03/2017
Começando a processar o bimestre 04/2017
Começando a processar o bimestre 05/2017
Começando a processar o bimestre 06/2017
Começando a proc

In [57]:
lista=['RECEITAS CORRENTES',
        'Impostos',
        'RECEITA PATRIMONIAL',
        'Contribuições Sociais',
        'RECEITA DE SERVIÇOS',
        'TRANSFERÊNCIAS CORRENTES',
        'OUTRAS RECEITAS CORRENTES']

lista2 = ['RECEITAS CORRENTES','Impostos', 'Taxas',
       'Contribuição de Melhoria', 
       'Contribuições Sociais', 
       'Exploração do Patrimônio Imobiliário do Estado',
       'Valores Mobiliários', 
       'Serviços Administrativos e Comerciais Gerais', 
       'Transferências da União e de suas Entidades',
       'Transferências dos Estados e do Distrito Federal e de suas Entidades',
       'Transferências dos Municípios e de suas Entidades',
       'Transferências de Outras Instituições Públicas',
       'Multas Administrativas, Contratuais e Judiciais',
       'Indenizações, Restituições e Ressarcimentos',
       'Bens, Direitos e Valores Incorporados ao Patrimônio Público',
       'Demais Receitas Correntes', 'Operações de Crédito - Mercado Interno',
       'Alienação de Bens Móveis']
df=pd.read_csv(path_download3)
df.set_index('Data',inplace=True)
df=df[df['Ano']>2018]
df_final=pd.DataFrame()

for selected_income  in lista2:
    filtered_df=df[df['Receitas Orçamentárias']==selected_income]
    filtered_df.fillna("0",inplace=True)
    filtered_df= arruma_num_rec(filtered_df)
    if len(filtered_df)>11:
        filtered_df.reset_index(inplace=True)
        filtered_df.reset_index(inplace=True)
        filtered_df['eh_par']=filtered_df['index'].apply(eh_par)
        filtered_df = filtered_df[filtered_df['eh_par']==1]
        filtered_df.set_index('Data',inplace=True)
        filtered_df.drop(['index','eh_par'],axis=1,inplace=True)
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências dos Estados e do Distrito Federal e de suas Entidades':
        filtered_df['Receitas Orçamentárias'] = ['Transf. dos Estados/DF']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências da União e de suas Entidades':
        filtered_df['Receitas Orçamentárias'] = ['Transf. da União']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Serviços Administrativos e Comerciais Gerais':
        filtered_df['Receitas Orçamentárias'] = ['Serviços Administrativos ']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Multas Administrativas, Contratuais e Judiciais':
        filtered_df['Receitas Orçamentárias'] = ['Multas']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências de Outras Instituições Públicas':
        filtered_df['Receitas Orçamentárias'] = ['Transf. Outras Inst.']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Operações de Crédito - Mercado Interno':
        filtered_df['Receitas Orçamentárias'] = ['Op. Crédito - Merc. Int.']*len(filtered_df['Receitas Orçamentárias'])
        
        
        
    
    filtered_df = pd.concat([filtered_df,ipca.iloc[18:]],axis=1,join='outer')
    filtered_df = aplica_inflacao_rec(filtered_df) 
    filtered_df.sort_values(by=['tempo'],inplace=True)
    df_final=df_final.append(filtered_df)
df_final.to_csv(path_download4)

/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-57-6a965f66c3d2>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [118]:
df_fim = df_final[df_final.index=='outubro 2020']
df_fim.sort_values('Receitas Realizadas no Bimestre (b)',inplace=True,ascending=False)
top10_value_temp = df_fim.iloc[:11]['Receitas Realizadas até o Bimestre (c)'].tolist()

top10_value_temp =['%.2f' % elem for elem in top10_value_temp ]
top10_value_temp = [float(item) for item in top10_value_temp]
top10_value=top10_value_temp[1:]
top10_name_temp= df_fim.iloc[:11]['Receitas Orçamentárias'].tolist()
top10_name_rec = top10_name_temp[1:]
top10_name=[elem.split()[0][:3] for elem in top10_name_rec ]

<ipython-input-118-031c4dd9b853>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fim.sort_values('Receitas Realizadas no Bimestre (b)',inplace=True,ascending=False)


In [151]:

filtered_df=df[df['Receitas Orçamentárias']=='Impostos']
filtered_df.fillna("0",inplace=True)
filtered_df= arruma_num_rec(filtered_df)
if len(filtered_df)>11:
    filtered_df.reset_index(inplace=True)
    filtered_df.reset_index(inplace=True)
    filtered_df['eh_par']=filtered_df['index'].apply(eh_par)
    filtered_df = filtered_df[filtered_df['eh_par']==1]
    filtered_df.set_index('Data',inplace=True)
    filtered_df.drop(['index','eh_par'],axis=1,inplace=True)
if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências dos Estados e do Distrito Federal e de suas Entidades':
    filtered_df['Receitas Orçamentárias'] = ['Transf. dos Estados/DF']*len(filtered_df['Receitas Orçamentárias'])
if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências da União e de suas Entidades':
    filtered_df['Receitas Orçamentárias'] = ['Transf. da União']*len(filtered_df['Receitas Orçamentárias'])
if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Serviços Administrativos e Comerciais Gerais':
    filtered_df['Receitas Orçamentárias'] = ['Serviços Administrativos ']*len(filtered_df['Receitas Orçamentárias'])
if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Multas Administrativas, Contratuais e Judiciais':
    filtered_df['Receitas Orçamentárias'] = ['Multas']*len(filtered_df['Receitas Orçamentárias'])
if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências de Outras Instituições Públicas':
    filtered_df['Receitas Orçamentárias'] = ['Transf. Outras Inst.']*len(filtered_df['Receitas Orçamentárias'])
if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Operações de Crédito - Mercado Interno':
    filtered_df['Receitas Orçamentárias'] = ['Op. Crédito - Merc. Int.']*len(filtered_df['Receitas Orçamentárias'])
        
        
        
    
filtered_df = pd.concat([filtered_df,ipca.iloc[18:]],axis=1,join='outer')
filtered_df = aplica_inflacao_rec(filtered_df) 
filtered_df.sort_values(by=['tempo'],inplace=True)

/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [130]:
parente = pathlib.Path().absolute()

path_download1 = Path(parente,"bases/piracicaba/receita_bimestres.csv")

df_final = pd.read_csv(path_download1)
df_fim = df_final[df_final['Unnamed: 0']=='outubro 2020']
df_fim.sort_values('Receitas Realizadas no Bimestre (b)',inplace=True,ascending=False)
top10_value_temp = df_fim.iloc[:11]['Receitas Realizadas no Bimestre (b)'].tolist()

top10_value_temp =['%.2f' % elem for elem in top10_value_temp ]
top10_value=top10_value_temp[1:]
top10_name_temp= df_fim.iloc[:11]['Receitas Orçamentárias'].tolist()
top10_name_rec = top10_name_temp[1:]
top10_name=[elem.split()[0][:3] for elem in top10_name_rec ]

<ipython-input-130-abecfa99f8e1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fim.sort_values('Receitas Realizadas no Bimestre (b)',inplace=True,ascending=False)


In [137]:
df_final.iloc[10]

Unnamed: 0                                      outubro 2020
Receitas Orçamentárias                    RECEITAS CORRENTES
Previsao Inicial                                    207625.0
Previsao Atualizada (a)                             207625.0
Receitas Realizadas no Bimestre (b)                  33375.2
% (b/a)                                                16.07
Receitas Realizadas até o Bimestre (c)             169787.32
% (c/a)                                                81.78
SALDO (a-c)                                         37837.68
Bimestre                                                   5
Ano                                                     2020
tempo                                             2020-10-20
base = ultimo periodo                                   0.86
Name: 10, dtype: object

## Receitas por ano

In [ ]:
ano = ['19','18','17','16']
pct='a'

for j in ano:
    print("Começando a processar o ano {}".format(j))
    if j == '18':
        file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/bal_orc_{}.pdf'.format(j)
    else:
        file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/bal_orc_{}.PDF'.format(j)
    todo = camelot.read_pdf(file,pages='1')
    
    if j== '19':
        receita_corrente = arruma_numero_unico(todo[0].df.iloc[1,8].split('\n')[0])
        receita_tributaria = arruma_numero_unico(todo[0].df.iloc[1,8].split('\n')[1])
        receita_patrimonial = arruma_numero_unico(todo[0].df.iloc[1,8].split('\n')[5])
        transferencias_gov = arruma_numero_unico(todo[0].df.iloc[1,6].split('\n')[9])
        outras_receitas = arruma_numero_unico(todo[0].df.iloc[1,6].split('\n')[14])
    elif j == '18':
        receita_corrente = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[0])
        receita_tributaria = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[1])
        receita_patrimonial = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[5])
        transferencias_gov = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[9])
        outras_receitas = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[15])
    else:
        receita_corrente = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[0])
        receita_tributaria = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[1])
        receita_patrimonial = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[5])
        transferencias_gov = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[10])
        outras_receitas = arruma_numero_unico(todo[0].df.iloc[1,3].split('\n')[13])
        
   

    p_receita_tributaria = (receita_tributaria/receita_corrente)*100
    p_receita_patrimonial = (receita_patrimonial/receita_corrente)*100
    p_transferencias_gov = (transferencias_gov /receita_corrente)*100
    p_outras_receitas = (outras_receitas /receita_corrente)*100
    
    dicionario = {'Receita Tributária':p_receita_tributaria,'Receita Patrimonial':p_receita_patrimonial,
                    'Transferencias Estado/Uniao':p_transferencias_gov,'Outras Receitas':p_outras_receitas }
    if type(pct)==str:
        pct = pd.DataFrame(list(dicionario.items()))
        pct['Data'] = '20'+ j
    else:
        transition=pd.DataFrame(list(dicionario.items()))
        transition['Data']= '20'+ j
        pct=pct.append(transition,ignore_index=True)
pct.columns=['Tipo Receita','%','Data']
pct.to_csv(path_download5, index = False)

## Despesa por bimestre

In [7]:
bimestre = ['01','02','03','04','05','06']
ano = ['2020','2019','2018','2017','2016']
despesa_orcamentaria=pd.DataFrame()
for j in ano:
    for i in bimestre:
        print("Começando a processar o bimestre {}/{}".format(i,j))
        if int(j) ==2020:
            if int(i)==6:
                continue
            else:
                pass
        
        file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        if j=='2016':
            todo=camelot.read_pdf(file,pages='2,3')
            despesa_orcamentaria_temp= todo[2].df.iloc[4:]
            despesa_orcamentaria_temp=pd.concat([despesa_orcamentaria_temp,todo[4].df.iloc[3:]])
        else:
            todo = camelot.read_pdf(file,pages='3')
            despesa_orcamentaria_temp=todo[1].df.iloc[2:]
            
        despesa_orcamentaria_temp.drop(10,axis=1,inplace=True)
        despesa_orcamentaria_temp.columns = ['Despesas Orçamentárias','DOTAÇÃO INICIAL (d)','DOTAÇÃO ATUALIZADA (e)','DESPESAS EMPENHADAS NO BIMESTRE','DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)','SALDO (g) =(e-f)','DESPESAS LIQUIDADAS NO BIMESTRE','DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)','SALDO (i) = (e-h)','DESPESAS PAGASATÉ O BIMESTRE(j)']
        despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
        despesa_orcamentaria_temp['Bimestre'] = ['{}'.format(i)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
        despesa_orcamentaria_temp['Ano'] = ['{}'.format(j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
        despesa_orcamentaria = pd.concat([despesa_orcamentaria_temp,despesa_orcamentaria])
        
despesa_orcamentaria = arruma_data(despesa_orcamentaria)
despesa_orcamentaria = despesa_orcamentaria[(despesa_orcamentaria['Despesas Orçamentárias']!='') & (despesa_orcamentaria['Despesas Orçamentárias']!='Despesas Orçamentárias')]
despesa_orcamentaria.to_csv(path_download6, index = True)


Começando a processar o bimestre 01/2020


/home/yan/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-7-a1c0df60916c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-7-a1c0df60916c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Começando a processar o bimestre 02/2020


/home/yan/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-7-a1c0df60916c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-7-a1c0df60916c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Começando a processar o bimestre 03/2020


...

Começando a processar o bimestre 05/2017


/home/yan/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-7-a1c0df60916c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-7-a1c0df60916c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Começando a processar o bimestre 06/2017


/home/yan/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-7-a1c0df60916c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-7-a1c0df60916c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Começando a processar o bimestre 01/2016
Começando a processar o bimestre 02/2016
Começando a processar o bimestre 03/2016
Começando a processar o bimestre 04/2016
Começando a processar o bimestre 05/2016
Começando a processar o bimestre 06/2016


In [58]:

lista=['DESPESAS CORRENTES',
        'PESSOAL E ENCARGOS SOCIAIS',
        'JUROS E ENCARGOS DA DÍVIDA',
        'OUTRAS DESPESAS CORRENTES',
        'INVESTIMENTOS',
        'AMORTIZAÇÃO DA DÍVIDA']
df=pd.read_csv(path_download6)
df.set_index('Data',inplace=True)

df_final=pd.DataFrame()
filtered_df =pd.DataFrame()
for selected_income  in lista:
    filtered_df=df[df['Despesas Orçamentárias']==selected_income]
    filtered_df=arruma_num_des(filtered_df)
    filtered_df = pd.concat([filtered_df,ipca],axis=1,join='outer')
    filtered_df = aplica_inflacao_des(filtered_df)
    filtered_df.sort_values(by=['tempo'],inplace=True)
    df_final=df_final.append(filtered_df)
df_final.to_csv(path_download7)

In [59]:
df_final

,Despesas Orçamentárias,DOTAÇÃO INICIAL (d),DOTAÇÃO ATUALIZADA (e),DESPESAS EMPENHADAS NO BIMESTRE,DESPESAS EMPENHADAS ATÉ O BIMESTRE (f),SALDO (g) =(e-f),DESPESAS LIQUIDADAS NO BIMESTRE,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h),SALDO (i) = (e-h),DESPESAS PAGASATÉ O BIMESTRE(j),Bimestre,Ano,tempo,base = ultimo periodo
fevereiro 2016,DESPESAS CORRENTES,1620.95,1637.70,684.12,684.12,953.58,212.58,212.58,1425.11,201.56,1,2016,2016-02-20,82.57
abril 2016,DESPESAS CORRENTES,1604.23,1623.70,375.80,1044.54,579.16,252.75,457.70,1166.00,450.36,2,2016,2016-04-20,83.43
junho 2016,DESPESAS CORRENTES,1586.26,1613.73,66.80,1099.64,514.08,281.68,734.25,879.47,724.53,3,2016,2016-06-20,84.37
agosto 2016,DESPESAS CORRENTES,1571.14,1599.46,147.58,1236.74,362.72,258.22,985.48,613.98,975.03,4,2016,2016-08-20,85.18
outubro 2016,DESPESAS CORRENTES,1565.81,1607.61,153.89,1386.44,221.18,259.59,1241.72,365.89,230.84,5,2016,2016-10-20,85.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fevereiro 2020,AMORTIZAÇÃO DA DÍVIDA,16.80,16.80,10.12,10.12,6.69,2.57,2.57,14.23,2.57,1,2020,2020-02-20,96.12
abril 2020,AMORTIZAÇÃO DA DÍVIDA,16.84,16.84,2.72,12.86,3.99,4.17,6.75,10.09,6.75,2,2020,2020-04-20,95.89
junho 2020,AMORTIZAÇÃO DA DÍVIDA,16.86,16.86,1.29,14.16,2.70,2.20,8.96,7.90,8.96,3,2020,2020-06-20,95.77
agosto 2020,AMORTIZAÇÃO DA DÍVIDA,16.76,17.54,-0.67,13.40,4.14,1.81,10.72,6.82,10.72,4,2020,2020-08-20,96.35


## Despesa por ano (%)

In [ ]:
anos=[2016,2017,2018,2019]
pct='a'
df=pd.read_csv(path_download7,',')
df.set_index('Unnamed: 0',inplace=True)
df_filtered_temp = df[df['Bimestre']==6]
for ano in anos:
    df_filtered = df_filtered_temp[df_filtered_temp['Ano']==ano]
    
    
    despesa_corrente = df_filtered.iloc[0,9]
    encargo_social = df_filtered.iloc[1,9]
    juros_divida = df_filtered.iloc[2,9]
    outras_despesas = df_filtered.iloc[3,9]

        
        
    
    p_encargo_social = (encargo_social/despesa_corrente)*100
    p_juros_divida = (juros_divida /despesa_corrente)*100
    p_outras_despesas = (outras_despesas /despesa_corrente)*100
    
    dicionario = {'Encargos Sociais':p_encargo_social,'Juros e encargos':juros_divida,
                    'Outras Despesas':p_outras_despesas }
    if type(pct)==str:
        pct = pd.DataFrame(list(dicionario.items()))
        pct['Data'] = ano
    else:
        transition=pd.DataFrame(list(dicionario.items()))
        transition['Data']= ano
        pct=pct.append(transition,ignore_index=True)
pct.columns=['Tipo Receita','%','Data']
pct.to_csv(path_download8,index = True)

## Despesa por função

In [129]:
datas=[3,5,7]
bimestre = ['01','02','03','04','05','06']
ano = ['2020','2019','2018']
despesa=pd.DataFrame()
for j in ano:
    for i in bimestre:
        print("Começando a processar o bimestre {}/{}".format(i,j))
        if int(j) ==2020:
            if int(i)==6:
                continue
            else:
                pass
        
        file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        
        todo = camelot.read_pdf(file,pages='7,8,9,10')
        despesa_temp=todo[1].df.iloc[3:]
        for d in datas:
            despesa_temp=despesa_temp.append(todo[d].df.iloc[3:])
        despesa_temp.columns = ['Despesas Orçamentárias','DOTAÇÃO INICIAL (d)','DOTAÇÃO ATUALIZADA (e)','DESPESAS EMPENHADAS NO BIMESTRE','DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)',('% (b/total b)'),'SALDO (g) =(e-f)','DESPESAS LIQUIDADAS NO BIMESTRE','DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)','% (d/total d)','SALDO (i) = (e-h)','DESPESAS PAGASATÉ O BIMESTRE(j)']
        despesa_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_temp.loc[:,'Despesas Orçamentárias'])
        despesa_temp['Bimestre'] = ['{}'.format(i)]*len(despesa_temp.loc[:,'Despesas Orçamentárias'])
        despesa_temp['Ano'] = ['{}'.format(j)]*len(despesa_temp.loc[:,'Despesas Orçamentárias'])
        despesa = pd.concat([despesa_temp,despesa])
        
despesa = arruma_data(despesa)
despesa = despesa[(despesa['Despesas Orçamentárias']!='') & (despesa['Despesas Orçamentárias']!='Despesas Orçamentárias')]
despesa.to_csv(path_download9, index = True)




Começando a processar o bimestre 01/2020
Começando a processar o bimestre 02/2020
Começando a processar o bimestre 03/2020
Começando a processar o bimestre 04/2020
Começando a processar o bimestre 05/2020
Começando a processar o bimestre 06/2020
Começando a processar o bimestre 01/2019
Começando a processar o bimestre 02/2019
Começando a processar o bimestre 03/2019
Começando a processar o bimestre 04/2019
Começando a processar o bimestre 05/2019
Começando a processar o bimestre 06/2019
Começando a processar o bimestre 01/2018
Começando a processar o bimestre 02/2018
Começando a processar o bimestre 03/2018
Começando a processar o bimestre 04/2018
Começando a processar o bimestre 05/2018
Começando a processar o bimestre 06/2018


In [60]:
listaTopo=['Legislativa',#Topo
    'Essencial à Justiça', #Topo
    'Administração',#Topo
    'Segurança Pública',#Topo
    'Assistência Social',#Topo
    'Previdência Social',#Topo
    'Saúde',#Topo
    'Educação',#Topo
    'Trabalho',#Topo
    'Cultura',#Topo
    'Direitos da Cidadania',#Topo
    'Urbanismo',#Topo
    'Habitação',#Topo
    'Saneamento',#Topo
    'Gestão Ambiental',#Topo
    'Ciência e Tecnologia',#Topo
    'Agricultura',#Topo
    'Turismo',#Topo
    'Transporte',#Topo
    'Desporto e Lazer',#Topo
    'Encargos Especiais'#Topo]
        ]
listaAdm=['Planejamento e Orçamento', 'Administração Financeira',
        'Controle Interno', 'Normatização e Fiscalização',
        'Tecnologia da Informação', 'Ordenamento Territorial',
        'Formação de Recursos Humanos', 'Administração de Receitas',
        'Administração de Concessões', 'Comunicação Social','FU04 - Administração Geral']

listSeg = ['Policiamento','Defesa Civil', 'Informação e Inteligência']

listaAss = ['Assistência ao Idoso','Assistência ao Portador de Deficiência',
        'Assistência à Criança e ao Adolescente','Assistência Comunitária']
listaSaude=['Atenção Básica', 'Assistência Hospitalar e Ambulatorial',
        'Suporte Profilático e Terapêutico', 'Vigilância Sanitária',
        'Vigilância Epidemiológica', 'Alimentação e Nutrição','FU10 - Administração Geral']
listaEdu=['Ensino Fundamental', 'Ensino Médio', 'Ensino Profissional',
        'Ensino Superior', 'Educação Infantil',
        'Educação de Jovens e Adultos', 'Educação Especial',
        'Educação Básica','FU12 - Demais Subfunções','FU12 - Administração Geral']
listaTrab=['Proteção e Benefícios ao Trabalhador', 'Relações de Trabalho',
        'Empregabilidade', 'Fomento ao Trabalho']       
listaCult = ['Patrimônio Histórico Artístico e Arqueológico',
        'Difusão Cultural'] 
listaUrb=['Infra-Estrutura Urbana', 'Serviços Urbanos',
        'Transportes Coletivos Urbanos']
listaSan=['Saneamento Básico Rural',
       'Saneamento Básico Urbano','FU17 - Administração Geral','FU17 - Demais Subfunções',]
listaGesAmb=[ 'Preservação e Conservação Ambiental', 'Controle Ambiental',
       'Recuperação de Áreas Degradadas', 'Recursos Hídricos',
        'Meteorologia','FU18 - Administração Geral' ]
listaAgr=['Abastecimento', 'Extensão Rural', 'Irrigação',
        'Promoção da Produção Agropecuária', 'Defesa Agropecuária','FU20 - Administração Geral']
listaDesp=['Desporto de Rendimento', 'Desporto Comunitário', 'Lazer','FU27 - Administração Geral']

listaListas = [listaTopo,listaAdm,listSeg,listaAss,listaSaude,listaEdu,
             listaTrab,listaCult,listaUrb,listaSan,listaGesAmb,listaAgr,listaDesp]

i=0
df=pd.read_csv(path_download9)
df.set_index('Data',inplace=True)

for lista  in listaListas:
    df_final=pd.DataFrame()
    filtered_df =pd.DataFrame()
    for item in lista:
        filtered_df=df[df['Despesas Orçamentárias']==item]
        filtered_df=arruma_num_des_fun(filtered_df)
        filtered_df.drop('DESPESAS PAGASATÉ O BIMESTRE(j)',axis=1,inplace=True)
        filtered_df.fillna(0,inplace=True)
        filtered_df = pd.concat([filtered_df,ipca.iloc[12:]],axis=1,join='outer')
        filtered_df = aplica_inflacao_des_fun(filtered_df)
        filtered_df.sort_values(by=['tempo'],inplace=True)
        df_final=df_final.append(filtered_df)
    i = i + 1
    df_final.to_csv(Path(path_download10,'{}.csv'.format(i)), index = True)
    

/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [62]:
df_final

,Despesas Orçamentárias,DOTAÇÃO INICIAL (d),DOTAÇÃO ATUALIZADA (e),DESPESAS EMPENHADAS NO BIMESTRE,DESPESAS EMPENHADAS ATÉ O BIMESTRE (f),% (b/total b),SALDO (g) =(e-f),DESPESAS LIQUIDADAS NO BIMESTRE,DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h),% (d/total d),SALDO (i) = (e-h),Bimestre,Ano,tempo,base = ultimo periodo
fevereiro 2018,Desporto de Rendimento,7.74,7.74,1.76,1.76,0.22,5315969.68,0.00,0.00,0.00,6884279.93,1,2018,2018-02-20,88.96
abril 2018,Desporto de Rendimento,7.72,7.86,0.47,2.23,0.22,5025794.86,0.42,0.43,0.09,6637663.23,2,2018,2018-04-20,89.23
junho 2018,Desporto de Rendimento,7.59,7.35,0.29,2.49,0.21,4415393.75,0.35,0.77,0.10,5970628.32,3,2018,2018-06-20,90.72
agosto 2018,Desporto de Rendimento,7.57,6.86,0.89,3.37,0.25,3171337.11,0.45,1.22,0.12,5124813.47,4,2018,2018-08-20,90.94
outubro 2018,Desporto de Rendimento,7.50,6.64,0.22,3.56,0.23,2832447.67,0.94,2.15,0.16,4121981.53,5,2018,2018-10-20,91.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fevereiro 2020,FU27 - Administração Geral,6.38,6.39,1.05,1.05,0.13,5136333.25,0.96,0.96,0.38,5221836.12,1,2020,2020-02-20,96.12
abril 2020,FU27 - Administração Geral,6.40,6.40,0.94,1.99,0.19,4232720.09,0.96,1.92,0.36,4298009.56,2,2020,2020-04-20,95.89
junho 2020,FU27 - Administração Geral,6.41,6.41,1.08,3.07,0.24,3202285.82,1.10,3.03,0.36,3239759.10,3,2020,2020-06-20,95.77
agosto 2020,FU27 - Administração Geral,6.37,6.37,0.91,3.96,0.28,2327104.06,0.84,3.85,0.35,2432459.94,4,2020,2020-08-20,96.35


In [61]:
df = pd.read_csv(Path(path_download10,'1.csv'))
df_last_year = df[df['Unnamed: 0']==df['Unnamed: 0'].iloc[-7]]
df_now = df[df['Unnamed: 0']==df['Unnamed: 0'].iloc[-1]]

df_now.set_index('Despesas Orçamentárias',inplace=True)
df_last_year.set_index('Despesas Orçamentárias',inplace=True)

df_pct = pd.DataFrame()
df_pct['Valor']= df_now['DESPESAS LIQUIDADAS NO BIMESTRE']
df_pct['Diferença']  = df_now['DESPESAS LIQUIDADAS NO BIMESTRE'] - df_last_year['DESPESAS LIQUIDADAS NO BIMESTRE']
df_pct['Diferença Absoluto'] = df_pct['Diferença'].apply(modulo)
df_pct['Porcentagem']  = (100*(df_now['DESPESAS LIQUIDADAS NO BIMESTRE'] - df_last_year['DESPESAS LIQUIDADAS NO BIMESTRE']))/df_last_year['DESPESAS LIQUIDADAS NO BIMESTRE']
df_pct['Porcentagem Absoluto'] = df_pct['Porcentagem'].apply(modulo)
df_pct.sort_values('Diferença Absoluto',inplace=True,ascending=False)
df_pct.drop(['Porcentagem Absoluto','Diferença Absoluto'],axis=1,inplace=True)



nome_1 = df_pct.index.tolist()[0]

pct_1 = round(df_pct['Porcentagem'].tolist()[0])
valor_1 = round(df_pct['Diferença'].tolist()[0])

nome_2 = df_pct.index.tolist()[1]
                     
pct_2= round(df_pct['Porcentagem'].tolist()[1])
valor_2= round(df_pct['Diferença'].tolist()[1])

nome_3 = df_pct.index.tolist()[2]
                     
pct_3= round(df_pct['Porcentagem'].tolist()[2])
valor_3= round(df_pct['Diferença'].tolist()[2])

nome_4 = df_pct.index.tolist()[3]
                     
pct_4 = round(df_pct['Porcentagem'].tolist()[3])
valor_4 = round(df_pct['Diferença'].tolist()[3])

nome_5 = df_pct.index.tolist()[4]
                     
pct_5 = round(df_pct['Porcentagem'].tolist()[4])
valor_5 = round(df_pct['Diferença'].tolist()[4])

nome_6 = df_pct.index.tolist()[5]
                     
pct_6 = round(df_pct['Porcentagem'].tolist()[5])
valor_6 = round(df_pct['Diferença'].tolist()[5])


In [63]:
valor_1

-8

In [15]:
df_valor[:10]['Valor'].tolist()

[68.03396418699154,
 60.27523887075458,
 36.499151561927,
 23.832552198090628,
 20.516048585068027,
 12.085058418812375,
 11.51522330801917,
 11.283402137104366,
 7.419059022154754,
 6.508699600800903]

In [16]:
df_valor[:10].index.tolist()

['Saúde',
 'Educação',
 'Saneamento',
 'Gestão Ambiental',
 'Previdência Social',
 'Urbanismo',
 'Administração',
 'Encargos Especiais',
 'Assistência Social',
 'Segurança Pública']

## Votos na Câmara

In [ ]:
todo[0].df.iloc[0,0].split('\n')

In [ ]:
base_df = pd.DataFrame()
num=['172','170','168','166','164','162','160']
for j in num:
    file = 'https://siave.camarapiracicaba.sp.gov.br/arquivo?Id=455{}'.format(j)
    todo = camelot.read_pdf(file,pages='all')
    for i in range(len(todo)):
        vereador = todo[i].df.iloc[0,0].split('\n')[15]
        if len(todo[i].df.iloc[0,0].split('\n'))==29:
            ementa = todo[i].df.iloc[0,0].split('\n')[17]
        elif len(todo[i].df.iloc[0,0].split('\n'))==30:
            ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]
        elif len(todo[i].df.iloc[0,0].split('\n'))==31:
            ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]
        elif len(todo[i].df.iloc[0,0].split('\n'))==32:
            ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]
        presentes = int(todo[i].df.iloc[0,1].split('\n')[0])
        ausentes = int(todo[i].df.iloc[0,1].split('\n')[2])
        sim = int(todo[i].df.iloc[1,1].split('\n')[1])
        nao = int(todo[i].df.iloc[1,1].split('\n')[0].split()[0])
        abst= int(todo[i].df.iloc[1,1].split('\n')[0].split()[1])
        status=todo[i].df.iloc[2,1]
        tempo_vot = todo[i].df.iloc[0,0].split('\n')[-3]
        dicionario = {'Propositor':vereador,'Ementa':ementa,'Presentes':presentes,'Ausentes':ausentes,
                      'Votos a Favor':sim,'Votos Contrários':nao, 'Abstenções':abst, 'Status':status,'Tempo de Votação':tempo_vot}
        base_df=base_df.append(dicionario,ignore_index=True)
base_df

In [ ]:
df_final=pd.DataFrame()
dicionario={'ADRIANA C. SGRIGNEIRO NUNES':27817,
           'ALDISA VIEIRA MARQUES': 27818,
           'ANDRÉ GUSTAVO BANDEIRA': 648,
           'ARY DE CAMARGO PEDROSO JUNIOR': 447,
           'CARLOS GOMES DA SILVA': 448,
           'DIRCEU ALVES DA SILVA': 659,
           'GILMAR ROTTA':665,
           'ISAC ALVES DE SOUZA': 27819,
           'JONSON S. DE OLIVEIRA':27820,
           'JOSÉ A. LONGATTO': 420,
           'JOSÉ FRANCISCO RONCATO': 27873,
           'JOSÉ MARCOS ABDALA':27821,
           'LAÉRCIO TREVISAN JR': 652,
           'LAIR BRAGA': 27822,
           'MATHEUS ANTONIO ERLER': 661,
           'NANCY A. F. THAME':27823,
           'OSVALDO A. SCHIAVOLIN':27824,
           'PAULO E. SEADE SERRA':671,
           'PAULO HENRIQUE PARANHOS RIBEIRO':651,
           'PAULO ROBERTO CAMPOS':668,
           'PEDRO MOTOITIRO KAWAI':666,
           'RERLISON T. DE REZENDE':27825,
           'RONALDO MOSCHINI SILVA':657,
           'WAGNER A. DE OLIVEIRA':27826,
           }
for ver in dicionario.items():
    print(ver[0], ver[1])
    url = 'https://siave.camarapiracicaba.sp.gov.br/Vereadores/Contagem/{}'.format(ver[1])
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df  = df_list[0].T.reset_index()
    df.drop('level_0',axis=1,inplace=True)
    df.columns=df.iloc[0]
    df=df.iloc[1:]
    df = df.melt(id_vars=['Tipo'], value_vars=df.columns[1:])
    df['Vereador']=ver[0]
    df_final=df_final.append(df,ignore_index=True)
df_final.columns=['Ano','Tipo','Valor','Vereador(a)']
df_final['Valor']=df_final['Valor'].apply(acha_string)

In [ ]:
df_final[(df_final['Tipo']=='Indicações') &(df_final['Ano']=='Total')]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'x': [0.05, 0.5], 'y': [0.15, 0]}))
fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'x': [0.05, 0.5], 'y': [0.15, 0]}))
fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 1}))
fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 1, 'column': 1}))


fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Força"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})

## Mapas Vereadores 2020

In [ ]:
mapa = pd.read_csv(path_download11,sep=',',encoding = "ISO-8859-1")
locais=pd.read_csv(path_download12)
locais['Seções2'] = locais['Seções'].apply(numeros_sessoes)
prefeito = mapa[mapa['DS_CARGO']=='PREFEITO']
vereador = mapa[mapa['DS_CARGO']=='VEREADOR']
pref_prim_turno = prefeito[prefeito['NR_TURNO']==1]
pref_seg_turno = prefeito[prefeito['NR_TURNO']==2]

dicionario={'André Bandeira':'ANDRE GUSTAVO BANDEIRA','Alessandra Protetora Animais':'Alessandra Bellucci'.upper(),
            'Paulo Camolesi':'Paulo Sergio Camolesi'.upper(),'Acácio Godoy':'ACÃCIO GERALDO SOUZA DE GODOY',
            'Ana Lúcia Pavão':'ANA LUCIA BATISTA PAVÃO','Paraná':'Aldisa Vieira Marques'.upper(),
            'Anilton Rissato':'Anilton Fernandes Rissato'.upper(), 'Ary Pedroso Jr':'Ary de Camargo Pedroso Junior'.upper(),
            'Cássio Luiz Fala Pira':'Cassio Luiz Barbosa'.upper(),'Fabricio Polezi':'FABRICIO JOSÃ RAETZ DE OLIVEIRA POLEZI',
            'Gilmar Rotta':'Gilmar Rotta'.upper(),'Gustavo Pompeo':'Gustavo Pompeo'.upper(),
            'Zezinho Pereira':'JOSÃ ANTONIO PEREIRA','Josef Borges':'Jose Everaldo Borges'.upper(),
            'Laércio Trevisan Jr':'Laercio Trevisan Junior'.upper(),'Paulo Henrique Ribeiro':'Paulo Henrique Paranhos Ribeiro'.upper(),
            'Paulo Roberto de Campos':'Paulo Roberto de Campos'.upper(),'Pedro Kawai':'Pedro Motoitiro Kawai'.upper(),
            'Rai de Almeida':'Raimunda Ferreira de Almeida'.upper(), 'Relinho':'Rerlison Teixeira de Rezende'.upper(),
            'Silvia Mandato Coletivo':'Silvia Maria Morales'.upper(),'Thiago Augusto Ribeiro':'Thiago Augusto Ribeiro'.upper(),
            'Wagnão':'Wagner Alexandre de Oliveira'.upper(),'Cimarinha':'CIMARA PEREIRA PRADA'
           }



ver_eleitos=pd.DataFrame()
ver_eleitos = vereador[vereador['NM_VOTAVEL'].isin(dicionario.values())]



ver_eleitos.loc[:,'Posicao'] = ver_eleitos.loc[:,'NR_SECAO'].apply(retorna_posicao)
ver_eleitos.loc[:,'Longitude'] = ver_eleitos.loc[:,'Posicao'].apply(lambda x: x.split(',')[1])
ver_eleitos.loc[:,'Latitude'] = ver_eleitos.loc[:,'Posicao'].apply(lambda x: x.split(',')[0])


df=pd.DataFrame()
for i in ver_eleitos['NR_SECAO'].unique():
    df_temp=ver_eleitos[ver_eleitos['NR_SECAO']==i]
    dici={'Seção':i,'Votos':df_temp.QT_VOTOS.sum()}
    df=df.append(dici,ignore_index=True)
ver_eleitos['Votos_Totais_Secao']=ver_eleitos['NR_SECAO'].apply(votos_totais_secao)
ver_eleitos.loc[:,'% votos da sessao']=(ver_eleitos.loc[:,'QT_VOTOS']/ver_eleitos.loc[:,'Votos_Totais_Secao'])*100

for item in dicionario.items():
    ver_eleito = ver_eleitos[ver_eleitos['NM_VOTAVEL']==item[1]]
    parent = pathlib.Path().absolute()
    path_download11 = Path(parent,"bases_brutas/{}/eleicao/vereadores/mapa/votacao.csv".format(cidade))
    path_download_especial = Path(path_download,'Site_Pira/Bases_Brutas/Camara/Vereadores/Vereadores/{}.csv'.format(item[0]))
    ver_eleito.to_csv(path_download_especial)

for nome in dicionario.keys():
    path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Camara/Vereadores/Vereadores/{}.csv'.format(nome))
    df=pd.read_csv(path_download_especial_1)
    nova=df.sort_values('% votos da sessao',ascending=False)
    nova = nova[nova['% votos da sessao']>7]
    coordenadas=[]
    for lat,lng in zip(nova.Latitude.values,nova.Longitude.values):
      coordenadas.append([lat,lng])
    mapa = folium.Map(location=[-22.732391682348542, -47.647584031127465],zoom_start=11)
    mapa.add_child(plugins.HeatMap(coordenadas))
    mapa.save('mapa_ver_{}.html'.format(nome))

## Mapas Prefeitos 1T/2020

In [8]:
mapa = pd.read_csv(path_download11,sep=',',encoding = "ISO-8859-1")
votacao = pd.read_csv(path_download12,sep=',',encoding = "ISO-8859-1")
prefeito = mapa[mapa['DS_CARGO']=='PREFEITO']
vereador = mapa[mapa['DS_CARGO']=='VEREADOR']
pref_prim_turno = prefeito[prefeito['NR_TURNO']==1]
pref_seg_turno = prefeito[prefeito['NR_TURNO']==2]


dicionario={'Nancy Thame':'NANCY APARECIDA FERRUZI THAME',
            'Francys Almeida':'FRANCYS ALMEIDA DA SILVA'.upper(),
            'Luciano Almeida':'Luciano Santos Tavares de Almeida'.upper(),
            'Professor Adelino':'Adelino Francisco de Oliveira'.upper(),
            'Mário Neto':'MÃ\x81RIO MEDEIROS NETO',
            'Zé Pedro':'JOSÃ\x89 PEDRO LEITE DA SILVA',
            'Érica Gorga':'Ã\x89RICA CRISTINA ROCHA GORGA'.upper(),
            'Barjas Negri':'BARJAS NEGRI'.upper(),
            'Carlito':'JOSE CARLOS LOPES DE SOUZA'.upper(),
            'Carolina Angelelli':'CAROLINA ANGELELLI'.upper(),
            'Coronel Adriana':'ADRIANA CRISTINA SGRIGNEIRO NUNES'.upper(),
            'Edvaldo Brito':'JOSE EDVALDO BRITO'.upper(),    
            }



locais=pd.read_csv(path_download12)
locais['Seções2'] = locais['Seções'].apply(numeros_sessoes)

pref_prim_turno.loc[:,'Posicao'] = pref_prim_turno.loc[:,'NR_SECAO'].apply(retorna_posicao)
pref_prim_turno.loc[:,'Longitude'] = pref_prim_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[1])
pref_prim_turno.loc[:,'Latitude'] = pref_prim_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[0])


df=pd.DataFrame()
for i in pref_prim_turno['NR_SECAO'].unique():
    df_temp=pref_prim_turno[pref_prim_turno['NR_SECAO']==i]
    dici={'Seção':i,'Votos':df_temp.QT_VOTOS.sum()}
    df=df.append(dici,ignore_index=True)
pref_prim_turno['Votos_Totais_Secao']=pref_prim_turno['NR_SECAO'].apply(votos_totais_secao)
pref_prim_turno.loc[:,'% votos da sessao']=(pref_prim_turno.loc[:,'QT_VOTOS']/pref_prim_turno.loc[:,'Votos_Totais_Secao'])*100

for nome in dicionario.items():
    pref_prim_turno_temp = pref_prim_turno[pref_prim_turno['NM_VOTAVEL']==nome[1]]
    path_download_especial = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/1T/{}.csv'.format(nome[0]))
    pref_prim_turno_temp.to_csv(path_download_especial)
    
for nome in dicionario.items():
    path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/1T/{}.csv'.format(nome[0]))
    df=pd.read_csv(path_download_especial_1)
    nova=df.sort_values('% votos da sessao',ascending=False).head(60)
    
    coordenadas=[]
    for lat,lng in zip(nova.Latitude.values,nova.Longitude.values):
      coordenadas.append([lat,lng])
    mapa = folium.Map(location=[-22.732391682348542, -47.647584031127465],zoom_start=11)
    mapa.add_child(plugins.HeatMap(coordenadas))        
    mapa.save('mapa_pref_{}.html'.format(nome[0]))

/home/yan/anaconda3/envs/Yan/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/yan/anaconda3/envs/Yan/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-8-03199f85dc39>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [ ]:
pref_prim_turno'NANCY APARECIDA FERRUZI THAME', 'BARJAS NEGRI', 'LUCIANO SANTOS TAVARES DE ALMEIDA'

In [7]:
pref_prim_turno[pref_prim_turno['NM_VOTAVEL']=='NANCY APARECIDA FERRUZI THAME']

,Unnamed: 0,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,...,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,Posicao,Longitude,Latitude,Votos_Totais_Secao,% votos da sessao
4,522745,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,7,2186,"-22.751091727766518, -47.65921187614776",-47.65921187614776,-22.751091727766518,573.00,1.22
31,522772,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,10,2186,"-22.751091727766518, -47.65921187614776",-47.65921187614776,-22.751091727766518,475.00,2.11
36,522777,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,14,2186,"-22.751091727766518, -47.65921187614776",-47.65921187614776,-22.751091727766518,542.00,2.58
59,522800,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,5,2186,"-22.751091727766518, -47.65921187614776",-47.65921187614776,-22.751091727766518,238.00,2.10
70,522811,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,8,2186,"-22.751091727766518, -47.65921187614776",-47.65921187614776,-22.751091727766518,488.00,1.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12638,614880,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,5,2160,"-22.754408914952744, -47.665043811082676",-47.665043811082676,-22.754408914952744,460.00,1.09
12651,614893,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,5,2178,"-22.78087217954171, -47.66667300313416",-47.66667300313416,-22.78087217954171,455.00,1.10
12671,614913,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,9,2178,"-22.78087217954171, -47.66667300313416",-47.66667300313416,-22.78087217954171,249.00,3.61
12689,614931,04/12/2020,15:37:43,2020,2,ELEIÃÃO ORDINÃRIA,1,426,ELEIÃÃES MUNICIPAIS 2020,15/11/2020,...,PREFEITO,43,NANCY APARECIDA FERRUZI THAME,7,2178,"-22.78087217954171, -47.66667300313416",-47.66667300313416,-22.78087217954171,485.00,1.44


In [ ]:
nome='BARJAS NEGRI'
path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/1T/{}.csv'.format(nome))
df=pd.read_csv(path_download_especial_1)
nova=df.sort_values('% votos da sessao',ascending=False)
nova

## Mapas Prefeitos 2T/2020

In [ ]:
mapa = pd.read_csv(path_download11,sep=',',encoding = "ISO-8859-1")
votacao = pd.read_csv(path_download12,sep=',',encoding = "ISO-8859-1")
prefeito = mapa[mapa['DS_CARGO']=='PREFEITO']
vereador = mapa[mapa['DS_CARGO']=='VEREADOR']
pref_prim_turno = prefeito[prefeito['NR_TURNO']==1]
pref_seg_turno = prefeito[prefeito['NR_TURNO']==2]


locais=pd.read_csv(path_download12)
locais['Seções2'] = locais['Seções'].apply(numeros_sessoes)

pref_seg_turno.loc[:,'Posicao'] = pref_seg_turno.loc[:,'NR_SECAO'].apply(retorna_posicao)
pref_seg_turno.loc[:,'Longitude'] = pref_seg_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[1])
pref_seg_turno.loc[:,'Latitude'] = pref_seg_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[0])


df=pd.DataFrame()
for i in pref_seg_turno['NR_SECAO'].unique():
    df_temp=pref_seg_turno[pref_seg_turno['NR_SECAO']==i]
    dici={'Seção':i,'Votos':df_temp.QT_VOTOS.sum()}
    df=df.append(dici,ignore_index=True)
pref_seg_turno['Votos_Totais_Secao']=pref_seg_turno['NR_SECAO'].apply(votos_totais_secao)
pref_seg_turno.loc[:,'% votos da sessao']=(pref_seg_turno.loc[:,'QT_VOTOS']/pref_seg_turno.loc[:,'Votos_Totais_Secao'])*100

for nome in pref_seg_turno.NM_VOTAVEL.unique():
    pref_seg_turno_temp = pref_seg_turno[pref_seg_turno['NM_VOTAVEL']==nome]
    path_download_especial = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/2T/{}.csv'.format(nome))
    pref_seg_turno_temp.to_csv(path_download_especial)
    
for nome in pref_seg_turno.NM_VOTAVEL.unique():
    path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/2T/{}.csv'.format(nome))
    df=pd.read_csv(path_download_especial_1)
    nova=df.sort_values('% votos da sessao',ascending=False).head(200)
    coordenadas=[]
    for lat,lng in zip(nova.Latitude.values,nova.Longitude.values):
      coordenadas.append([lat,lng])
    mapa = folium.Map(location=[-22.732391682348542, -47.647584031127465],zoom_start=14)
    mapa.add_child(plugins.HeatMap(coordenadas))        
    mapa.save('mapa_pref_{}.html'.format(nome))

In [1]:
import os
cwd = os.getcwd()
cwd

'/home/yan/Site_Yan/InformaPira/Notebook'

# CAGED

Baixar o arquivo "tabelas" em http://pdet.mte.gov.br/novo-caged